In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures
import datetime 
import pytz
from pytz import timezone
import tarfile
from numpy import load
import os
from sklearn.manifold import TSNE
import h5py
from sklearn.decomposition import PCA as sklearnPCA

/Users/marin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Putting all the feature vectors in a dataframe

In [4]:
d = np.load('/Users/marin/redhook/embeddings/sonycnode-b827eb208550.sonyc/2019-06-01/00/1559361628.56.npz')

In [5]:
d.files

['embedding', 'timestamps']

In [6]:
d['embedding']

array([[2.0931249, 1.1152985, 3.2679663, ..., 2.3449025, 1.4089253,
        2.4516468],
       [1.9170014, 1.479332 , 3.0609157, ..., 2.5557625, 1.5059618,
        2.823496 ],
       [2.0005364, 2.1422334, 2.4985769, ..., 2.5461915, 1.8356978,
        2.8371768],
       ...,
       [1.9174784, 1.605869 , 3.1989174, ..., 3.0292304, 1.6785302,
        2.3574908],
       [1.9913956, 1.5095303, 3.0157228, ..., 2.0276167, 1.5995463,
        2.4592245],
       [1.8192018, 1.6748338, 3.4832704, ..., 2.9015133, 1.5057862,
        2.7135656]], dtype=float32)

In [7]:
d['embedding']

array([[2.0931249, 1.1152985, 3.2679663, ..., 2.3449025, 1.4089253,
        2.4516468],
       [1.9170014, 1.479332 , 3.0609157, ..., 2.5557625, 1.5059618,
        2.823496 ],
       [2.0005364, 2.1422334, 2.4985769, ..., 2.5461915, 1.8356978,
        2.8371768],
       ...,
       [1.9174784, 1.605869 , 3.1989174, ..., 3.0292304, 1.6785302,
        2.3574908],
       [1.9913956, 1.5095303, 3.0157228, ..., 2.0276167, 1.5995463,
        2.4592245],
       [1.8192018, 1.6748338, 3.4832704, ..., 2.9015133, 1.5057862,
        2.7135656]], dtype=float32)

In [8]:
# df = pd.DataFrame(data=d['embedding'])
# df.head()

In [9]:
os.path.basename('/Users/marin/redhook/embeddings/sonycnode-b827eb208550.sonyc/2019-06-01/00/1559361628.56.npz')
os.path.basename('/Users/marin/redhook/embeddings/sonycnode-b827eb208550.sonyc/2019-06-01/00/1559361628.56.npz')[:10]


'1559361628'

Loops through all files in hour 0 of 6-01 and adds the timestamps to an array

In [8]:
timestamps_6_01_00 = np.empty(385)
data_6_01_00 = np.empty((385, 512))
count_6_01_00 = 0
day_01_00 = '/Users/marin/redhook/embeddings/sonycnode-b827eb208550.sonyc/2019-06-01/00'
for filename in sorted(os.listdir(day_01_00)):
    day_01_00_embedding = np.load(day_01_00 + '/' + filename)['embedding']
    for sec in range(11):
        timestamps_6_01_00[count_6_01_00] = str(int(os.path.basename(filename)[:10]) + sec)
        data_6_01_00[count_6_01_00] = day_01_00_embedding[sec]
        count_6_01_00 += 1
    #count_6_01_00 += 1
count_6_01_00

385

In [9]:
timestamps_6_01_00_dt = np.empty(385, dtype = datetime.datetime)
for i in range(385):
    j = timestamps_6_01_00[i]
    dt = datetime.datetime.utcfromtimestamp(j)
    dt = pytz.UTC.localize(dt)
    dt = dt.astimezone(pytz.timezone('US/Eastern'))
    timestamps_6_01_00_dt[i] = dt

In [10]:
df = pd.DataFrame(data_6_01_00)
df.insert(0, "Time", timestamps_6_01_00_dt, True)
df.head()

,Time,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,2019-06-01 00:00:28-04:00,2.093125,1.115299,3.267966,2.600699,4.386734,0.614799,1.508289,1.911545,1.631085,...,2.329870,2.693596,2.634851,2.617877,2.263398,2.064028,1.211207,2.344903,1.408925,2.451647
1,2019-06-01 00:00:29-04:00,1.917001,1.479332,3.060916,4.766980,3.649807,1.389943,1.774898,2.440066,1.192924,...,3.151508,2.616026,3.144591,1.897207,2.344439,2.753301,1.542656,2.555763,1.505962,2.823496
2,2019-06-01 00:00:30-04:00,2.000536,2.142233,2.498577,4.384803,3.449889,2.402933,1.792420,2.963485,1.735846,...,3.351357,2.549209,2.706580,2.142213,2.041494,2.246689,1.084968,2.546191,1.835698,2.837177
3,2019-06-01 00:00:31-04:00,1.856699,1.582061,2.593662,3.662261,3.377521,1.645816,1.405100,2.683466,1.198949,...,2.656243,2.167135,3.039538,1.907364,1.839635,2.889525,0.995547,2.107294,2.030357,3.319138
4,2019-06-01 00:00:32-04:00,1.896574,1.591261,2.830297,4.182943,3.380369,1.186396,1.081441,2.225049,1.322107,...,3.404805,2.849727,2.882296,2.141266,1.937293,2.918287,1.119735,2.479955,1.673045,2.788664


Same as above, but for the whole day

In [11]:
timestamps_6_01 = np.empty(9537)
data_6_01 = np.empty((9537, 512))
count_6_01 = 0
day_01 = '/Users/marin/redhook/embeddings/sonycnode-b827eb208550.sonyc/2019-06-01'
for dir_hour in range(24):
    day_hour= day_01 + '{:02d}'.format(dir_hour)
    for filename_1 in sorted(os.listdir(day_01 + '/' + '{:02d}'.format(dir_hour))):
#         print(filename_1 + " " + dirname_1)
        day_01_embedding = np.load(day_01 + '/' + '{:02d}'.format(dir_hour) + '/' + filename_1)['embedding']
        for sec_1 in range(11):
            timestamps_6_01[count_6_01] = str(int(os.path.basename(filename_1)[:10]) + sec_1)
            data_6_01[count_6_01] = day_01_embedding[sec_1]
            count_6_01 += 1

In [12]:
timestamps_6_01_dt = np.empty(9537, dtype = datetime.datetime)
for i in range(9537):
    j = timestamps_6_01[i]
    dt = datetime.datetime.utcfromtimestamp(j)
    dt = pytz.UTC.localize(dt)
    dt = dt.astimezone(pytz.timezone('US/Eastern'))
    timestamps_6_01_dt[i] = dt

In [13]:
day_01_df = pd.DataFrame(data_6_01)
day_01_df.insert(0, "Time", timestamps_6_01_dt, True)
day_01_df.tail()

,Time,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
9532,2019-06-01 23:58:04-04:00,1.256165,1.629046,2.780099,3.507525,3.552612,1.586547,1.327846,1.559000,1.311527,...,2.482014,2.412203,3.680104,1.573512,2.017706,2.750537,1.031552,2.343161,1.674596,2.657019
9533,2019-06-01 23:58:05-04:00,1.417175,1.574511,2.730569,3.654880,3.966511,1.488733,1.438324,1.466175,1.571570,...,2.324653,2.153596,3.366831,1.473867,2.310448,1.681509,1.161637,2.041306,1.735808,2.352880
9534,2019-06-01 23:58:06-04:00,1.552803,1.666663,2.471927,3.355394,3.596159,1.405086,1.119313,2.417684,1.275138,...,2.867661,2.022134,3.044971,1.708414,2.361706,1.428441,1.265338,2.088488,1.468976,1.975224
9535,2019-06-01 23:58:07-04:00,1.641850,1.511594,2.891329,3.710870,3.666551,1.675946,1.685685,1.670958,1.503443,...,2.805249,2.329516,3.452280,1.393967,2.183595,1.786110,1.081113,2.265397,1.614978,3.059705
9536,2019-06-01 23:58:08-04:00,1.791041,1.531177,3.659442,4.340420,3.490789,1.915224,1.365899,1.161948,1.287995,...,2.205489,2.218124,3.668745,2.568184,2.063264,2.244980,2.381623,3.098542,1.628558,2.714632


Same as above, but for the whole month!

In [14]:
timestamps_6 = np.empty(284504)
data_6 = np.empty((284504, 512))
count_6 = 0
month_1 = '/Users/marin/redhook/embeddings/sonycnode-b827eb208550.sonyc/2019-06-'
for day in range(1, 31):
    month_day_1 = month_1 + '{:02d}'.format(day)
    for hour in range(24):
        for filename_minute in sorted(os.listdir(month_day_1 + '/' + '{:02d}'.format(hour))):
#         print(filename_1 + " " + dirname_1)
            day_embedding = np.load(month_day_1 + '/' + '{:02d}'.format(hour) + '/' + filename_minute)['embedding']
            for sec_2 in range(11):
                timestamps_6[count_6] = str(int(os.path.basename(filename_minute)[:10]) + sec_2)
                data_6[count_6] = day_embedding[sec_2]
                count_6 += 1

In [15]:
timestamps_6_dt = np.empty(284504, dtype = datetime.datetime)
for i in range(284504):
    j = timestamps_6[i]
    dt = datetime.datetime.utcfromtimestamp(j)
    dt = pytz.UTC.localize(dt)
    dt = dt.astimezone(pytz.timezone('US/Eastern'))
    timestamps_6_dt[i] = dt

In [16]:
month_df = pd.DataFrame(data_6)
month_df.insert(0, "Time", timestamps_6_dt, True)
month_df.tail()

,Time,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
284499,2019-06-30 23:58:38-04:00,1.037169,1.214468,2.732894,4.681792,3.452725,1.165627,0.872909,1.126685,1.352854,...,2.108679,1.879616,3.362580,0.860649,2.188544,1.349597,0.881527,2.117487,1.664466,2.010612
284500,2019-06-30 23:58:39-04:00,1.003718,0.938289,2.587343,4.757359,3.431142,1.083679,0.391370,1.560856,1.530192,...,2.026901,1.937228,3.361410,0.904966,2.201618,1.295208,1.118922,2.150769,1.752120,1.948045
284501,2019-06-30 23:58:40-04:00,0.912182,1.099503,2.560506,4.931278,3.442858,1.239084,0.496955,1.396896,1.081517,...,1.957695,2.171432,3.119542,0.930831,2.330969,1.241590,0.722740,2.240005,1.749369,1.772667
284502,2019-06-30 23:58:41-04:00,0.948775,1.792279,2.286294,4.537268,3.647916,0.966004,0.512753,1.302830,0.836954,...,1.893391,2.169309,2.976476,0.817437,2.314738,1.709069,0.564410,2.197109,1.751422,1.892172
284503,2019-06-30 23:58:42-04:00,1.791041,1.620573,3.800530,4.498526,3.514423,0.992465,1.289594,1.831819,0.866663,...,1.820580,2.009623,2.831064,2.525323,2.121424,2.075847,2.337930,3.178445,1.829217,2.589327


# All four sensors, for the whole month!

In [17]:
# timestamps_6 = np.empty(284504)
# data_6 = np.empty((284504, 512))
# count_6 = 0
# month_1 = '/Users/marin/redhook/embeddings/sonycnode-b827eb208550.sonyc/2019-06-'
# for day in range(1, 31):
#     month_day_1 = month_1 + '{:02d}'.format(day)
#     for hour in range(24):
#         for filename_minute in sorted(os.listdir(month_day_1 + '/' + '{:02d}'.format(hour))):
# #         print(filename_1 + " " + dirname_1)
#             day_embedding = np.load(month_day_1 + '/' + '{:02d}'.format(hour) + '/' + filename_minute)['embedding']
#             for sec_2 in range(11):
#                 timestamps_6[count_6] = str(int(os.path.basename(filename_minute)[:10]) + sec_2)
#                 data_6[count_6] = day_embedding[sec_2]
#                 count_6 += 1

In [20]:
# time_sensor_dict = {}
# for sensor in os.listdir('/Users/marin/redhook/embeddings'):
#     if sensor[0] == 's':
#         for day in range(1, 31):
#             month_day_1 = month_1 + '{:02d}'.format(day)
#             for hour in range(24):
#                 for filename_minute in sorted(os.listdir(month_day_1 + '/' + '{:02d}'.format(hour))):
# #         print(filename_1 + " " + dirname_1)
#                     day_embedding = np.load(month_day_1 + '/' + '{:02d}'.format(hour) + '/' + filename_minute)['embedding']
#                     for sec_2 in range(11):
#                         time_sensor_dict{} = str(int(os.path.basename(filename_minute)[:10]) + sec_2)
#                         timestamps_6[count_6] = 
#                         data_6[count_6] = day_embedding[sec_2]
#                         count_6 += 1

In [3]:
#3403950
sensor_timestamps = np.empty(3388858)
sensor_data = np.empty((3388858, 512))
sensor_count = 0
sensor_id = []

In [4]:
for sensor in os.listdir('/Users/marin/redhook/embeddings'):
    if sensor[0] == 's':
        month = '/Users/marin/redhook/embeddings/' + sensor
        for sensor_day in sorted(os.listdir(month)):
#             sensor_month_day = month + '{:02d}'.format(sensor_day)
            for sensor_hour in sorted(os.listdir(month + '/' + sensor_day)):
                for sensor_filename_minute in sorted(os.listdir(month + '/' + sensor_day + '/' + sensor_hour)):
#         print(filename_1 + " " + dirname_1)
                    sensor_day_embedding = np.load(month + '/' + sensor_day + '/' + sensor_hour + '/' + sensor_filename_minute)['embedding']
                    for sec_3 in range(11):
                        sensor_timestamps[sensor_count] = str(int(os.path.basename(sensor_filename_minute)[:10]) + sec_3)
                        sensor_id.append(sensor)
                        sensor_data[sensor_count] = sensor_day_embedding[sec_3]
                        sensor_count += 1

# HDF5

In [ ]:
h5 = h5py.File()
do you rstuf here

h5.close()

In [5]:
sensor_data.shape

(3388858, 512)

In [5]:
with h5py.File('sound_data.hdf5', 'w') as h5:
    d = h5.create_dataset('sound_data',
                          (3403950,),
                          dtype=[('timestamp', 'f8'),
                                 ('sensor_id', 'S128'),
                                 ('feature_vector', 'f4', (512,))
                                ],
                            chunks=True,
                            maxshape=(3403950 * 514,))
#     print(d[0])
    for idx in range(sensor_data.shape[0]):
        d[idx] = (sensor_timestamps[idx], sensor_id[idx], sensor_data[idx])
#        d[idx] = (sensor_month_df.loc[idx]['timestamp'], sensor_month_df.loc[idx]['sensor_id'], sensor_month_df.loc[idx]['feature_vector'])
        

In [3]:
h5 = h5py.File('sound_data.hdf5', 'r')

In [4]:
d = h5['sound_data']

In [6]:
d['feature_vector'][0]

array([1.8989118 , 1.679023  , 3.260314  , 3.2904801 , 3.4298534 ,
       0.9699534 , 1.6788019 , 1.1441466 , 1.2537551 , 5.2354174 ,
       2.0002575 , 3.6110406 , 1.7126136 , 2.6451123 , 3.1612499 ,
       2.0423524 , 1.961672  , 1.2752104 , 2.5082521 , 1.7135687 ,
       2.6164181 , 5.1630054 , 3.361977  , 2.8295515 , 3.563549  ,
       2.2390609 , 1.6374077 , 1.3829205 , 0.6565321 , 2.3014553 ,
       3.683128  , 3.7668376 , 2.7939203 , 1.5972351 , 1.563678  ,
       3.4779754 , 2.0474377 , 1.5181998 , 1.9800779 , 2.7542572 ,
       0.7846313 , 3.7893026 , 2.7794828 , 1.0161594 , 1.4670504 ,
       1.644988  , 3.3318229 , 2.8648162 , 1.5065205 , 2.0327754 ,
       2.4176247 , 2.4241128 , 1.9271816 , 3.4241147 , 2.3759897 ,
       3.4184496 , 2.3706336 , 2.0048776 , 1.1496958 , 2.6474304 ,
       1.9019674 , 2.0197034 , 2.51896   , 1.2705511 , 2.64906   ,
       2.6732016 , 1.9028354 , 1.9185073 , 2.0812216 , 2.2369907 ,
       2.0206795 , 1.6939882 , 1.176535  , 2.2587354 , 1.74416

In [7]:
# with h5py.File('sound_data_dt.hdf5', 'w') as h5:
#     d_dt = h5.create_dataset('sound_data',
#                           (3388858,),
#                           dtype=[('timestamp', ''),
#                                  ('sensor_id', 'S128'),
#                                  ('feature_vector', 'f4', (512,))
#                                 ],
#                             chunks=True,
#                             maxshape=(3388858 * 514,))
# #     print(d[0])
#     for idx in range(sensor_data.shape[0]):
#         d[idx] = (sensor_timestamps[idx], sensor_id[idx], sensor_data[idx])

Can't do this because h5py doesn't support datetime! 

In [55]:
a = 0
for index in d:
    timestamp = d[index][0]
    dt = datetime.datetime.utcfromtimestamp(timestamp)
    dt = pytz.UTC.localize(dt)
    dt = dt.astimezone(pytz.timezone('US/Eastern'))
    d[index][0] = dt
#     a += 1

TypeError: '>=' not supported between instances of 'numpy.ndarray' and 'numpy.bytes_'

In [1]:
#d['feature_vector'][1]

In [2]:
d_sample = []
#d_sample.append(np.random.choice(d['feature_vector'], 100000, replace = False))
sample_nums = np.random.choice(range(3388858), 1000, replace = False)
for num in sample_nums:
    d_sample.append(d['feature_vector'][num])

NameError: name 'np' is not defined

In [1]:
# pca = sklearnPCA(n_components=0)
# fitted = pca.fit(d['feature_vector'])

In [19]:
for sensor in os.listdir('/Users/marin/redhook/embeddings'):
    if sensor[0] == 's':
        month = '/Users/marin/redhook/embeddings/' + sensor + '/2019-06-'
        for sensor_day in range(1, 31):
            sensor_month_day = month + '{:02d}'.format(sensor_day)
            for sensor_hour in range(24):
                for sensor_filename_minute in sorted(os.listdir(sensor_month_day + '/' + '{:02d}'.format(sensor_hour))):
#         print(filename_1 + " " + dirname_1)
#                     sensor_day_embedding = np.load(sensor_month_day + '/' + '{:02d}'.format(sensor_hour) + '/' + sensor_filename_minute)['embedding']
                    for sec_3 in range(11):
#                         sensor_timestamps[sensor_count] = str(int(os.path.basename(sensor_filename_minute)[:10]) + sec_3)
#                         sensor_id.append(sensor)
#                         sensor_data[sensor_count] = sensor_day_embedding[sec_3]
                        sensor_count += 1

FileNotFoundError: [Errno 2] No such file or directory: '/Users/marin/redhook/embeddings/sonycnode-b827ebe3b72c.sonyc/2019-06-01/03'

In [20]:
import glob

In [11]:
# list(glob.glob('/Users/marin/redhook/embeddings/*/*/*/*.npz')[:100])

In [22]:
import re

In [24]:
pat = '.+(sonycnode-.+sonyc)/([0-9\-]+)/\d\d/([0-9\.]+)\.npz'

In [26]:
mat = re.match(pat, '/Users/marin/redhook/embeddings/sonycnode-b827ebc178d2.sonyc/2019-06-01/03/1559373430.89.npz')

In [31]:
mat.groups()

('sonycnode-b827ebc178d2.sonyc', '2019-06-01', '1559373430.89')

In [ ]:
sensor_timestamps_dt = np.empty(3388858, dtype = datetime.datetime)
for i in range(3388858):
    j = sensor_timestamps[i]
    dt = datetime.datetime.utcfromtimestamp(j)
    dt = pytz.UTC.localize(dt)
    dt = dt.astimezone(pytz.timezone('US/Eastern'))
    sensor_timestamps_dt[i] = dt

In [ ]:
sensor_month_df = pd.DataFrame(sensor_data)
sensor_month_df.insert(0, "timestamp", sensor_timestamps_dt, True)
sensor_month_df.insert(1, "sensor_id", sensor_id, True)

In [24]:
sensor_month_df.head()

,Time,Sensor ID,0,1,2,3,4,5,6,7,...,502,503,504,505,506,507,508,509,510,511
0,2019-06-30 00:00:01-04:00,sonycnode-b827ebc178d2.sonyc,2.147170,1.811349,3.613613,2.166399,4.279818,0.729798,1.608264,1.417915,...,1.750993,1.508241,3.169659,2.898161,2.278058,1.516454,1.035957,2.341614,1.320728,2.381136
1,2019-06-30 00:00:02-04:00,sonycnode-b827ebc178d2.sonyc,2.040531,2.060741,3.132879,2.738560,4.217388,0.842049,1.683079,1.319852,...,1.903533,1.715690,3.752447,1.350698,2.159956,1.114236,0.726913,2.318805,1.309860,2.208788
2,2019-06-30 00:00:03-04:00,sonycnode-b827ebc178d2.sonyc,2.267295,2.052840,2.916851,2.829883,4.362580,0.831749,1.355194,1.243675,...,2.753550,1.553952,3.651077,1.572992,2.514829,0.903902,0.893332,2.409671,1.410028,1.933796
3,2019-06-30 00:00:04-04:00,sonycnode-b827ebc178d2.sonyc,2.010005,1.933367,3.155593,2.752398,4.131149,0.656648,1.105359,1.103871,...,1.777061,1.956103,3.265976,0.953201,2.251938,1.454855,0.632706,2.196253,1.450269,1.885378
4,2019-06-30 00:00:05-04:00,sonycnode-b827ebc178d2.sonyc,1.672005,2.230400,3.199084,3.315724,4.048234,0.911971,1.671144,1.321853,...,2.120324,1.409174,3.188677,1.562922,2.046928,1.010944,0.571363,2.849014,1.930400,1.990249


In [76]:
sensors = np.array([sensor_timestamps_dt, sensor_id, sensor_data])
sensors[0].shape

(3403950,)

In [25]:
sensor_count_dict = {}
for sensor in sorted(os.listdir('/Users/marin/redhook/embeddings')):
    if sensor[0] == 's':
        sensor_count = 0
        month = '/Users/marin/redhook/embeddings/' + sensor + '/2019-06-'
        for sensor_day in range(1, 31):
            sensor_month_day = month + '{:02d}'.format(day)
            for sensor_hour in range(24):
                for sensor_filename_minute in sorted(os.listdir(sensor_month_day + '/' + '{:02d}'.format(sensor_hour))):
#         print(filename_1 + " " + dirname_1)
#                     sensor_day_embedding = np.load(sensor_month_day + '/' + '{:02d}'.format(sensor_hour) + '/' + sensor_filename_minute)['embedding']
                    for sec_3 in range(11):
                        sensor_count += 1
        sensor_count_dict[sensor] = sensor_count

In [26]:
#Total number of feature vectors in each sensor 
sensor_count_dict

{'sonycnode-b827eb208550.sonyc': 287760,
 'sonycnode-b827eb491436.sonyc': 1418010,
 'sonycnode-b827ebc178d2.sonyc': 1419000,
 'sonycnode-b827ebe3b72c.sonyc': 279180}

In [ ]:
# sensor_month_df_sample = sensor_month_df.sample(frac=0.00294117647, replace=True, random_state=1)

In [ ]:
sensor_month_df.loc[0]['Time']

In [14]:
# h5 = h5py.File('sound_data.hdf5', 'w')
#     d = h5.create_dataset('sound_data',
#                           (3403950,),
#                           dtype=[('timestamp', 'f8'),
#                                  ('sensor_id', 'S128'),
#                                  ('feature_vector', 'f4', (512,))
#                                 ],
#                             chunks=True,
#                             maxshape=(3403950 * 514,))
    
#     for sensor in os.listdir('/Users/marin/redhook/embeddings'):
#     if sensor[0] == 's':
#         month = '/Users/marin/redhook/embeddings/' + sensor + '/2019-06-'
#         for sensor_day in range(1, 31):
#             sensor_month_day = month + '{:02d}'.format(sensor_day)
#             for sensor_hour in range(24):
#                 for sensor_filename_minute in sorted(os.listdir(sensor_month_day + '/' + '{:02d}'.format(sensor_hour))):
# #         print(filename_1 + " " + dirname_1)
#                     sensor_day_embedding = np.load(sensor_month_day + '/' + '{:02d}'.format(sensor_hour) + '/' + sensor_filename_minute)['embedding']
#                     for sec_3 in range(11):
#                         sensor_timestamps[sensor_count] = str(int(os.path.basename(sensor_filename_minute)[:10]) + sec_3)
#                         sensor_id.append(sensor)
#                         sensor_data[sensor_count] = sensor_day_embedding[sec_3]
#                         sensor_count += 1

In [8]:
count_6_01_00 = 0
day_01_00 = '/Users/marin/redhook/embeddings/sonycnode-b827eb208550.sonyc/2019-06-01/00'
for filename in sorted(os.listdir(day_01_00)):
    day_01_00_embedding = np.load(day_01_00 + '/' + filename)['embedding']
    for sec in range(11):
        timestamps_6_01_00[count_6_01_00] = str(int(os.path.basename(filename)[:10]) + sec)
        data_6_01_00[count_6_01_00] = day_01_00_embedding[sec]
        count_6_01_00 += 1
    #count_6_01_00 += 1
count_6_01_00

385

In [9]:
timestamps_6_01_00_dt = np.empty(385, dtype = datetime.datetime)
for i in range(385):
    j = timestamps_6_01_00[i]
    dt = datetime.datetime.utcfromtimestamp(j)
    dt = pytz.UTC.localize(dt)
    dt = dt.astimezone(pytz.timezone('US/Eastern'))
    timestamps_6_01_00_dt[i] = dt

In [10]:
df = pd.DataFrame(data_6_01_00)
df.insert(0, "Time", timestamps_6_01_00_dt, True)
df.head()

,Time,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,2019-06-01 00:00:28-04:00,2.093125,1.115299,3.267966,2.600699,4.386734,0.614799,1.508289,1.911545,1.631085,...,2.329870,2.693596,2.634851,2.617877,2.263398,2.064028,1.211207,2.344903,1.408925,2.451647
1,2019-06-01 00:00:29-04:00,1.917001,1.479332,3.060916,4.766980,3.649807,1.389943,1.774898,2.440066,1.192924,...,3.151508,2.616026,3.144591,1.897207,2.344439,2.753301,1.542656,2.555763,1.505962,2.823496
2,2019-06-01 00:00:30-04:00,2.000536,2.142233,2.498577,4.384803,3.449889,2.402933,1.792420,2.963485,1.735846,...,3.351357,2.549209,2.706580,2.142213,2.041494,2.246689,1.084968,2.546191,1.835698,2.837177
3,2019-06-01 00:00:31-04:00,1.856699,1.582061,2.593662,3.662261,3.377521,1.645816,1.405100,2.683466,1.198949,...,2.656243,2.167135,3.039538,1.907364,1.839635,2.889525,0.995547,2.107294,2.030357,3.319138
4,2019-06-01 00:00:32-04:00,1.896574,1.591261,2.830297,4.182943,3.380369,1.186396,1.081441,2.225049,1.322107,...,3.404805,2.849727,2.882296,2.141266,1.937293,2.918287,1.119735,2.479955,1.673045,2.788664
